<a href="https://colab.research.google.com/github/kaisarmasum/Generative-AI-Use-Case-Summarize-Dialogue/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install transformers

In [ ]:

import cv2
img = cv2.imread('/content/download.png', 0)


# shape prints the tuple (height,weight,channels)
print("image shape = ", img.shape)

# img will be a numpy array of the above shape
print("image array = ", img)

print("pixel at index (5,5): ", img[5][5])

image shape =  (168, 300)
image array =  [[159 159 159 ... 161 161 161]
 [159 159 159 ... 161 161 161]
 [159 159 159 ... 161 161 161]
 ...
 [159 159 159 ... 161 161 161]
 [159 159 159 ... 161 161 161]
 [159 159 159 ... 161 161 161]]
pixel at index (5,5):  159


In [ ]:
# this is pixel of 0th row and 0th column
print(img[1][1])

159


In [ ]:
import torch
from torch import nn
from io import StringIO
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

data = '/content/all_stocks_5yr.csv'

# Use StringIO to convert the data string into a file-like object for pd.read_csv
df = pd.read_csv(data)

In [ ]:
df = df[:3000]

In [ ]:
df.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [ ]:
df.dropna()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.1200,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.0100,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.5100,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.9400,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.9600,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
2995,2014-12-31,160.64,162.6940,159.26,159.28,468081,AAP
2996,2015-01-02,160.85,162.5000,157.47,158.56,509983,AAP
2997,2015-01-05,158.17,158.9875,155.90,156.47,803635,AAP
2998,2015-01-06,156.56,157.2900,153.90,156.36,872305,AAP


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

# Load the data
# Assuming the data is stored in a CSV file named 'your_data.csv'
df = pd.read_csv('/content/all_stocks_5yr.csv')
df = df[:100]

# Normalize price and volume data
normalized_cols = ["open", "high", "low", "close", "volume"]
df[normalized_cols] = (df[normalized_cols] - df[normalized_cols].mean()) / df[normalized_cols].std()

# Split into features and target
X = df[["open", "high", "low", "volume"]]
y = df["close"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Create sequences using indices directly
def create_sequences(indices, seq_length):
    sequences, labels = [], []
    for i in range(len(indices) - seq_length):
        seq = indices[i:i + seq_length]
        label = indices[i + seq_length]  # Fix the label indexing
        sequences.append(seq)
        labels.append(label)
    return torch.tensor(sequences), torch.tensor(labels)

seq_length = 10  # Adjust the sequence length based on your requirements
train_indices = range(len(X_train))
test_indices = range(len(X_train), len(X_train) + len(X_test))
train_x, train_y = create_sequences(train_indices, seq_length)
test_x, test_y = create_sequences(test_indices, seq_length)

# DataLoader for batching and shuffling
batch_size = 64
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Update the input_size to be the vocabulary size
input_size = max(test_x.max().item() + 1, len(set(train_x.flatten().tolist() + test_x.flatten().tolist())))

class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=2):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=2, num_encoder_layers=n_layers)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, src):
        src = self.embedding(src)
        src = src.permute(1, 0, 2)
        output = self.transformer(src, src)
        output = self.fc(output.permute(1, 0, 2)[-1, :, :]).squeeze()
        return output.view(-1, 1)

# Instantiate the model and set hyperparameters
hidden_size = 512  # Adjust based on your requirements
n_layers = 32  # Number of transformer layers

model = TransformerModel(input_size, hidden_size, n_layers)

# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_x, train_y = train_x.to(device), train_y.to(device)

# Training
criterion = nn.MSELoss()  # You can change this to another loss function like nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_epochs = 10

for epoch in range(num_epochs):
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x.to(device)).squeeze()
        loss = nn.MSELoss()(torch.log1p(outputs), torch.log1p(batch_y.view(-1, 1).to(device)))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

    scheduler.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have t

Epoch [1/10], Loss: 0.5271
Epoch [2/10], Loss: 0.9185
Epoch [3/10], Loss: 0.5416
Epoch [4/10], Loss: 0.3270
Epoch [5/10], Loss: 0.3384
Epoch [6/10], Loss: 0.4292
Epoch [7/10], Loss: 0.0646
Epoch [8/10], Loss: 0.2951
Epoch [9/10], Loss: 0.2722
Epoch [10/10], Loss: 0.7894


In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import numpy as np

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(test_x)

    test_outputs = test_outputs.squeeze()
    test_y = test_y[:len(test_outputs)]


    test_outputs = test_outputs.float()
    test_y = test_y.float()


    rmse = np.sqrt(mean_squared_error(test_y, test_outputs))
    print(f'Test RMSE: {rmse:.4f}')

    mae = mean_absolute_error(test_y, test_outputs)

    print(f'Mean Absolute Error (MAE): {mae:.4f}')

Test RMSE: 56.2026
Mean Absolute Error (MAE): 56.1292
